In [7]:
# --- Import Libraries ---
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [9]:
# --- 1. Define Dataset Path ---
dataset_path = "Plant_Dataset"  # <--- **ADJUST THIS IF YOUR FOLDER NAME IS DIFFERENT**

In [11]:
# --- 2. Define Image Transformations ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for CNNs like ResNet
    transforms.ToTensor(),           # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize using ImageNet stats
])

In [13]:
# --- 3. Load Dataset ---
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [15]:
# --- 4. Split Dataset ---
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [17]:
# --- 5. Create DataLoaders ---
batch_size = 32  # <--- **YOU CAN ADJUST THE BATCH SIZE HERE**
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [19]:
# --- 6. Print Dataset Info ---
print(f"Total images: {len(dataset)}")
print(f"Training images: {len(train_dataset)}, Testing images: {len(test_dataset)}")
print(f"Class names: {dataset.classes}")
num_classes = len(dataset.classes)
print(f"Number of classes: {num_classes}")

Total images: 19178
Training images: 15342, Testing images: 3836
Class names: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Background_without_leaves', 'Blueberry___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)']
Number of classes: 17


In [21]:
# --- 7. Load Pretrained Model (ResNet18) ---
model = models.resnet18(pretrained=True)

# --- 8. Modify the Final Fully Connected Layer ---
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes) # Use the actual number of classes

# --- 9. Move the Model to GPU if Available ---
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

C:\Users\jared\anaconda3\envs\DeepLearning_PlantClass_39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jared\anaconda3\envs\DeepLearning_PlantClass_39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cpu


In [23]:
# --- 10. Define Loss Function and Optimizer ---
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001 # <--- **YOU CAN ADJUST THE LEARNING RATE HERE**
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
# --- 11. Train the Model ---
def train_model(model, train_loader, criterion, optimizer, num_epochs=5): # <--- **ADJUST THE NUMBER OF EPOCHS HERE**
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        model.train() # Set the model to training mode

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # Zero the gradients

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward() # Perform backpropagation
            optimizer.step() # Update the weights

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / total_samples
        epoch_accuracy = correct_predictions / total_samples
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_accuracy:.4f}')

In [ ]:
# Train the model
num_epochs = 5 # You can change this here as well
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [24]:
# --- 12. Evaluate the Model ---
def evaluate_model(model, test_loader, criterion):
    model.eval() # Set the model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad(): # Disable gradient calculation during evaluation
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = running_loss / total_samples
    test_accuracy = correct_predictions / total_samples
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [26]:
# Evaluate the model
evaluate_model(model, test_loader, criterion)

Test Loss: 0.1162, Test Accuracy: 0.9638


In [28]:
# --- 13. Save the Trained Model ---
torch.save(model.state_dict(), 'plant_classification_model.pth')
print("Trained model saved as plant_classification_model.pth")

Trained model saved as plant_classification_model.pth


In [5]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# Load your saved model
model = models.resnet18(pretrained=False) # Or with weights if you prefer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))
model.load_state_dict(torch.load('plant_classification_model.pth'))
model.eval()

# Load and preprocess a new image
image_path = 'C:/Users/jared/anaconda3/envs/DeepLearning_PlantClass_39/image_berry.jpg'
image = Image.open(image_path).convert('RGB')
transform_single = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = transform_single(image).unsqueeze(0) # Add a batch dimension

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class_index = torch.argmax(probabilities).item()
    predicted_class_name = dataset.classes[predicted_class_index]
    confidence = probabilities[predicted_class_index].item()

print(f"Predicted class: {predicted_class_name} (Confidence: {confidence:.4f})")

NameError: name 'models' is not defined